In [1]:
import numpy as np
import os
import scipy.io

In [2]:
def loadData(folderName):
    data = []
    labels = []
    allFiles = os.listdir(folderName)
    for file in allFiles:
        loadedFile = scipy.io.loadmat(folderName+'/'+file)
        data.append(loadedFile['data'])
        labels.append(loadedFile['labels'])

    return np.array(data), np.array(labels)

In [3]:
data, labels = loadData("../Datasets/data_preprocessed_matlab")

In [4]:
# subject * video * channel * data
print(data.shape)
# subject * video * labels
print(labels.shape)

(32, 40, 40, 8064)
(32, 40, 4)


In [5]:
def getSpecificChannels():
    channels = np.array(['Fp1', 'AF3', 'F3', 'F7', 'FC5', 'FC1', 'C3', 'T7', 'CP5', 'CP1', 'P3', 'P7', 'PO3', 'O1', 'Oz',
                        'Pz', 'Fp2', 'AF4', 'Fz', 'F4', 'F8', 'FC6', 'FC2', 'Cz', 'C4', 'T8', 'CP6', 'CP2', 'P4', 'P8', 'PO4', 'O2'])
    selectedChannel = np.array(
        ['AF3', 'F3', 'F7', 'FC5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6', 'F8', 'F4', 'AF4'])
#     selectedChannel = np.array(['Fp1', 'Fp2', 'F3', 'F4'])
    return np.where(np.in1d(channels, selectedChannel))[0]

In [6]:
channelIndices = getSpecificChannels()
channelIndices

array([ 1,  2,  3,  4,  7, 11, 13, 17, 19, 20, 21, 25, 29, 31])

In [7]:
# Select 14 channel
selectedChannelData = data[:,:,channelIndices,:]
print(selectedChannelData.shape)

(32, 40, 14, 8064)


In [8]:
# Remove first 20 second of video
sampleRate = 128
removed20SecData = selectedChannelData[:,:,:,sampleRate*20:sampleRate*60]

In [9]:
removed20SecData.shape

(32, 40, 14, 5120)

In [29]:
processedData = np.zeros((32,160,14,1280))
# processedData = np.zeros((32,160,4,1280))
# processedData = np.zeros((32,80,4,2560))
parts = [removed20SecData[:,:,:,:sampleRate*10],
         removed20SecData[:,:,:,sampleRate*10:sampleRate*20],
         removed20SecData[:,:,:,sampleRate*20:sampleRate*30],
         removed20SecData[:,:,:,sampleRate*30:]
        ]
# parts = [removed20SecData[:,:,:,:sampleRate*20],
#          removed20SecData[:,:,:,sampleRate*20:]
#         ]

for s in range(32):
    for v in range(40):
        for i in range(4):
            processedData[s,(v * 4)+i, :,:] = parts[i][s,v,:,:]
            
processedLabels = np.repeat(labels,4, axis=1)

In [30]:
print(processedData.shape)
print(processedLabels.shape)

(32, 160, 14, 1280)
(32, 160, 4)


In [39]:
processedLabels[25]

array([[6.03, 5.04, 9.  , 6.  ],
       [6.03, 5.04, 9.  , 6.  ],
       [6.03, 5.04, 9.  , 6.  ],
       [6.03, 5.04, 9.  , 6.  ],
       [7.  , 7.03, 9.  , 8.1 ],
       [7.  , 7.03, 9.  , 8.1 ],
       [7.  , 7.03, 9.  , 8.1 ],
       [7.  , 7.03, 9.  , 8.1 ],
       [9.  , 8.04, 9.  , 9.  ],
       [9.  , 8.04, 9.  , 9.  ],
       [9.  , 8.04, 9.  , 9.  ],
       [9.  , 8.04, 9.  , 9.  ],
       [7.05, 7.01, 9.  , 7.08],
       [7.05, 7.01, 9.  , 7.08],
       [7.05, 7.01, 9.  , 7.08],
       [7.05, 7.01, 9.  , 7.08],
       [4.97, 1.  , 9.  , 1.  ],
       [4.97, 1.  , 9.  , 1.  ],
       [4.97, 1.  , 9.  , 1.  ],
       [4.97, 1.  , 9.  , 1.  ],
       [7.  , 5.04, 8.08, 7.96],
       [7.  , 5.04, 8.08, 7.96],
       [7.  , 5.04, 8.08, 7.96],
       [7.  , 5.04, 8.08, 7.96],
       [4.94, 5.04, 9.  , 7.03],
       [4.94, 5.04, 9.  , 7.03],
       [4.94, 5.04, 9.  , 7.03],
       [4.94, 5.04, 9.  , 7.03],
       [8.24, 8.17, 8.01, 9.  ],
       [8.24, 8.17, 8.01, 9.  ],
       [8.

In [31]:
subjectCount = processedData.shape[0]
videoCount = processedData.shape[1]
channelCount = processedData.shape[2]

In [32]:
valenceLabels = ((processedLabels[:, :, 0] >= 5) * 1)
arousalLabels = ((processedLabels[:, :, 1] >= 5) * 1)
dominanceLabels = ((processedLabels[:, :, 2] >= 5) * 1)
likingLabels = ((processedLabels[:, :, 3] >= 5) * 1)

In [33]:
valenceLabels.shape

(32, 160)

In [23]:
dominanceLabels.shape

(32, 40)

In [22]:
def normalize(data):
    return (data - data.mean() )/ data.std()

def getNDifference(data,n):
    return data[n:] - data[:-n]

def extractStatisticFeature(data,subjectCount,videoCount,channelCount):
    
    subjectFeatures = []
    for s in range(subjectCount):
        
        videoFeatures = []
        for v in range(videoCount):
            
            channelFeatures = []
            for ch in range(channelCount):
                
                item = data[s, v, ch]
                normalized = normalize(item)
                
                mean = item.mean()
                std = item.std()
                firstDif = np.mean(abs(getNDifference(item,1)))
                secDif = np.mean(abs(getNDifference(item,2)))
                normFirstDif = firstDif/item.std()
                normSecDef = secDif/item.std()
                
                features = np.array([mean,std,firstDif,secDif,normFirstDif,normSecDef])
                channelFeatures.append(features)
            
            videoFeatures.append(channelFeatures)
            
        subjectFeatures.append(videoFeatures)
        
    return np.array(subjectFeatures)

In [23]:
sFeatures = extractStatisticFeature(processedData,subjectCount,videoCount,channelCount)
print(sFeatures.shape)

(32, 40, 14, 6)


In [24]:
sFeatures = sFeatures.reshape(sFeatures.shape[0],sFeatures.shape[1],sFeatures.shape[2]*sFeatures.shape[3])
print(sFeatures.shape)

(32, 40, 84)


In [25]:
valenceLabels.shape

(32, 40)

In [26]:
import pickle
filename="statisticalFeatures40video.dat"
with open(filename,"wb") as f:
    pickle.dump(sFeatures, f)

In [27]:
with open(filename,"rb") as f:
    features = pickle.load(f)

In [28]:
features.shape

(32, 40, 84)

In [34]:
import pickle
filename="Dataset/dominanceLabels.dat"
with open(filename,"wb") as f:
    pickle.dump(dominanceLabels, f)

In [35]:
with open(filename,"rb") as f:
    dominanceLabels = pickle.load(f)

In [36]:
dominanceLabels.shape

(32, 160)

In [37]:
dominanceLabels

array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 1, 1, 1]])

In [16]:
arousalLabels.shape

(32, 160)

In [19]:
filename="Dataset/likingLabels.dat"
with open(filename,"wb") as f:
    pickle.dump(likingLabels, f)